In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point

## Make a napari viewer...

In [2]:
v = napari.Viewer()

## Load cell x gene data from 1 dataset

In [46]:
all_cells = pd.read_csv("/Users/brian/Downloads/smFISH_mapping (1)/smFISH_expressionMatrix.csv")

metadata = pd.read_csv("/Users/brian/Downloads/smFISH_mapping (1)/smFISH_metaData.csv")
metadata["x_um"] = metadata["x"]*1e3
metadata["y_um"] = metadata["y"]*1e3

In [47]:
all_cells = all_cells.T.rename(columns = all_cells["Unnamed: 0"])

all_cells = all_cells.drop("Unnamed: 0")


all_cells.index.name= "cellID"


metadata = metadata.set_index("cellID")

In [48]:
all_cells = all_cells.merge(metadata, on = "cellID")

### Add the cells as points, double-plotting cells with high Rorb, Cux2 if available

In [50]:
cells = v.add_points(all_cells[["x_um","y_um"]], symbol='o', name = "all", opacity = 0.8, blending = "translucent")
colorlist = ["magenta", "green","yellow","cyan", "blue", "orange"]
for ii, gene_to_plot in enumerate(["Cux2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in all_cells:
        cux2condition = all_cells[gene_to_plot]>0.5*all_cells[gene_to_plot].median()
        cux2cells = v.add_points(all_cells[cux2condition][["x_um","y_um"]], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= "black", size = 10,
                                 face_color= colorlist[ ii % len(colorlist)] , blending = "translucent", opacity = 0.3)


### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire column as rectangle
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [51]:
v.layers

[<Points layer 'all' at 0x12cd33f10>,
 <Points layer 'Cux2+' at 0x12cb934d0>,
 <Points layer 'Fezf2+' at 0x12d326810>,
 <Points layer 'Rorb+' at 0x12d5296d0>,
 <Shapes layer 'Shapes' at 0x12d707750>]

In [52]:
v.layers[-1].data

[array([[-1887.5990672 , -2286.80536833],
        [ -846.4396003 , -1551.17834779],
        [-1618.3180774 ,  -458.71088631],
        [-2659.4775443 , -1194.33790685]]),
 array([[-1865.69121122, -2265.07801297],
        [-1742.97664837, -2178.35013097],
        [-2504.78129225, -1100.44390638],
        [-2627.4958551 , -1187.17178838]]),
 array([[-1744.80471456, -2168.98522852],
        [-1505.5479902 , -2000.76022336],
        [-2255.98031504,  -933.46360047],
        [-2495.2370394 , -1101.68860563]]),
 array([[-1504.51910557, -1990.85887509],
        [-1287.84057632, -1834.85033403],
        [-2045.49768122,  -782.54879945],
        [-2262.17621047,  -938.55734051]]),
 array([[-1295.06502424, -1840.32945847],
        [-1062.55301538, -1676.42322362],
        [-1812.73978189,  -612.23288168],
        [-2045.25179076,  -776.13911653]]),
 array([[-1057.21424803, -1682.87121315],
        [ -861.17678461, -1543.97827347],
        [-1615.33244221,  -479.54143102],
        [-1811.36990564,

In [53]:
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in v.layers[0].data] for ii,p in enumerate(v.layers[-1].data[:-1])}
column_points = v.layers[-1].data[-1]

In [ ]:
# ok:
# 1. make a couple functions to do this
#.  #. do a spatially partitioned search based on bounding box of polygon
# 2. throw annotation into separate vector matched to point cloud
#. 

In [54]:
# depth coordinate calculation

def depth_coordinate(points_xy, column_axis_points):
    """ 
    calculate the projection along the column axis vector
    assumes that the pia is the first point in column_axis_points
    """
    column_vec = np.diff(column_axis_points, axis = 0)
    column_vec = column_vec.T/np.linalg.norm(column_vec,2)
    
    output_array = np.row_stack(
        [np.dot(a_point-column_axis_points[0],column_vec) for a_point in points_xy])
    return output_array

In [55]:
all_cells["depth_um"] = depth_coordinate(all_cells[["x_um","y_um"]].values, column_points)
all_cells["layer"] = "outside_VISp"
layer_list = ["VISp", "VISp_I", "VISp_II/III", "VISp_IV", "VISp_V","VISp_VI"]
for l_key in is_in_layer.keys():
    all_cells.loc[is_in_layer[l_key],"layer"] = layer_list[l_key]
    

In [13]:
is_in_layer.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6])

In [56]:
all_cells.to_csv("Allen_smFISH_with_depth.csv")

In [57]:
all_cells

,Alcam,Chodl,Cux2,Fezf2,Foxp2,Gad2,Galnt14,Grin3a,Kcnip4,Kcnk2,...,x,y,z,area,layerData,experiment,x_um,y_um,depth_um,layer
cellID,,,,,,,,,,,,,,,,,,,,,
Cell_1,212,12,29,133,13,180,75,54,9,46,...,-2.010159,-0.538822,0.004136,0.000581,0,MouseVISp_AI,-2010.158777,-538.821628,898.241746,outside_VISp
Cell_2,70,5,3,179,6,6,141,21,5,30,...,-1.984237,-0.720638,0.004227,0.000484,0,MouseVISp_AI,-1984.236740,-720.637885,813.052193,VISp
Cell_3,169,4,113,6,4,121,4,3,4,28,...,-1.725721,-1.984365,0.004266,0.000640,0,MouseVISp_AI,-1725.720789,-1984.364680,284.516980,VISp_II/III
Cell_4,133,2,84,1,4,63,2,0,1,18,...,-2.340063,-0.630326,0.004250,0.000531,0,MouseVISp_AI,-2340.062786,-630.326065,577.039612,outside_VISp
Cell_5,27,1,6,54,42,93,0,0,1,70,...,-1.908918,-0.177810,0.004256,0.000567,0,MouseVISp_AI,-1908.917654,-177.810141,1191.328852,outside_VISp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cell_1962,47,2,30,2,1,0,2,1,2,2,...,-1.937050,-1.744502,0.004263,0.000129,0,MouseVISp_AI,-1937.049915,-1744.502433,253.149381,VISp_II/III
Cell_1976,55,0,16,0,0,0,0,0,0,2,...,-1.565728,-1.975440,0.004252,0.000109,0,MouseVISp_AI,-1565.728195,-1975.439780,419.576585,VISp_II/III
Cell_2010,20,2,24,1,2,1,1,1,2,1,...,-2.468299,-0.711818,0.004267,0.000118,0,MouseVISp_AI,-2468.299305,-711.818023,425.354601,outside_VISp
